In [1]:
# General
import os
import sys
import funcy as fp

# Visualization / Presentation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML, display

import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd

# Carregar, além de atualizar frequentemente, código personalizado disponível em ../src
%load_ext autoreload 
%autoreload 2
sys.path.append(os.path.abspath(os.path.pardir))
from src import settings
from src.utils.notebooks import display_side_by_side

# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
sns.set(rc={'figure.figsize':(25,10)})
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)

## Introdução

Este notebook tem como objetivo permitir o registro visual dos experimentos feitos e oferecer um modo de recuperar um experimento e analisar os resultados em mais detalhes.
Os princípios usados para recuperar o modelo e as funções de pré-processamento usadas aqui também podem ser usadas para "produtizar" o classificador e o recomendador -- com as ressalvas já feitas anteriormente sobre modos melhores de implementar o pipeline para o ambiente de produção e para os modos de processamento unitário ou em lote.

## Recuperação do melhor resultado

A seguir, considerando o primeiro experimento feito, recupera-se o melhor resultado a partir da métrica do F1 médio entre as categorias.

In [2]:
EXPERIMENT_ID = '0'

mlflow_client = MlflowClient()

best_experiments_result = [
    mlflow.search_runs(experiment_ids=[experiment_id], 
                       max_results=100, 
                       order_by=['metrics.F1 DESC'], 
                       filter_string='attributes.status="FINISHED"')
    for experiment_id in [EXPERIMENT_ID]
]

best_results = pd.concat(best_experiments_result, axis=0)

Dada uma execução, tem-se a lista de resultados individuais dos modelos e a execução agregadora, sem nome de modelo (*arams.model_name*), que mantém as informações do experimento como um todo (e.g., funções e parâmetros de pré-processamento).

In [3]:
columns_to_show = ['experiment_name', 'tags.mlflow.runName', 'run_id', 'experiment_id', 'params.model_name',
                   'metrics.f1', 'metrics.precision', 'metrics.recall', 'metrics.training_time']

(best_results
 .assign(experiment_name=lambda f: f['experiment_id'].apply(lambda id: mlflow_client.get_experiment(id).name))
 [columns_to_show] 
 .head()
)

,experiment_name,tags.mlflow.runName,run_id,experiment_id,params.model_name,metrics.f1,metrics.precision,metrics.recall,metrics.training_time
0,01_SupervisedClassification,01_2_Acceptable Set (Title and Tags Emb. and Sim. + Price)_CB,2f2ef8c238084aa0b850353d2fa98e57,0,CB,0.775664,0.755899,0.802780,24.559202
1,01_SupervisedClassification,01_2_Acceptable Set (Title and Tags Emb. and Sim. + Price)_MLP,763ffc6e95b94cddbe45cf6ff41126d3,0,MLP,0.783002,0.786404,0.780058,20.365059
2,01_SupervisedClassification,01_1_Acceptable Set (Title and Tags Emb. and Sim. + Price)_CB,0e645800d18940a5873cc24dc3683842,0,CB,0.766290,0.748748,0.790108,24.579118
3,01_SupervisedClassification,01_1_Acceptable Set (Title and Tags Emb. and Sim. + Price)_MLP,fa6926f4d36d4024b946cb7345007b63,0,MLP,0.767018,0.761639,0.773711,40.689214
4,01_SupervisedClassification,01_0_Acceptable Set (Title and Tags Emb. and Sim. + Price)_CB,bcfaedc85a7e4ced8f10b6b89d584292,0,CB,0.778817,0.760689,0.801611,24.679635


## Restauração de Experimentos

A partir da escolha de uma execução individual, é possível restaurar os elementos utilizados na experimentação para aplicá-los aos dados.

In [4]:
# Restaura o experimento pelo maior valor de F1 Médio (entre as classes)
best_experiments_results = mlflow.search_runs(experiment_ids=[EXPERIMENT_ID], max_results=100, order_by=['metrics.f1 DESC'], filter_string='attributes.status="FINISHED"')
best_experiment = (best_experiments_results
 .loc[lambda f: ~f['params.model_name'].isna()]
 .sort_values(by='metrics.f1', ascending=False)
 .drop_duplicates(['params.model_name'])
 .loc[lambda f: f['params.model_name'] == 'CB']
 .iloc[0]
)

# Recupera o ID do artefato para recuperar modelos e recursos persistidos para o experimento
artifact_uri = best_experiment["artifact_uri"]

# Mostra as principais informações da execução
display(HTML('<h4>Melhor resultado:</h4>'))
for name, key in [('Run ID', 'run_id'),
                  ('Model Name', 'params.model_name'),
                  ('Average F1', 'metrics.f1'),
                  ('Average Precision', 'metrics.precision'),
                  ('Average Recall', 'metrics.recall'),
                 ]:
    display(HTML(f'<li><strong>{name}</strong>: {best_experiment[key]}</li>'))

# Recupera o pacote de funções e parâmetros de pré-processamento dos dados
preprocessing_model_path = os.path.join(best_experiment['artifact_uri']
                                        .replace(best_experiment['run_id'],
                                                 best_experiment['tags.mlflow.parentRunId']),
                                        'log',
                                        'preprocessing_model')
preprocessing_model = mlflow.pyfunc.load_model(preprocessing_model_path)

# Recupera o modelo treinado
model = mlflow.sklearn.load_model(f'{artifact_uri}/model')

# Recupera o Label Encoder, caso seja preciso avaliar o modelo
label_encoder_path = os.path.join(best_experiment['artifact_uri'].replace(best_experiment['run_id'], best_experiment['tags.mlflow.parentRunId']), 'label_encoder')
label_encoder_model = mlflow.sklearn.load_model(label_encoder_path)

Para validar o funcionamento da restauração do modelo, parte dos dados de treinamento são recuperados para uma avaliação.

In [5]:
columns_to_read = ['title', 'concatenated_tags', 'price', 'weight', 'express_delivery', 'minimum_quantity', 'category', 'creation_date']
frame = (pd
         .read_csv(os.path.join(settings.DATA_PATH, 'interim', 'training.csv'), usecols=columns_to_read)         
         .drop_duplicates('category')
         .sort_values(by='category')
        )

Tendo os dados, é possível reprocessar *labels*, *features* e fazer a inferência.

In [6]:
# Processa os dados para inferência
features = preprocessing_model.predict(frame)

# Realiza a inferência
frame['pred'] = model.predict(features)

# Codificar labels
frame['label'] = label_encoder_model.transform(frame.category)

display_side_by_side([frame, 
                      pd.DataFrame(features).describe().T.head(10)], 
                     ['Dados Recuperados e Predição', 
                      f'Features (10 de {features.shape[1]})'])